In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DX = [0, 1, 1, 1, 0, -1, -1, -1]
DY = [-1, -1, 0, 1, 1, 1, 0, -1]

def generate_neighbour(img, y, x):
    h, w = img.shape
    p2 = img[y-1][x] if y > 0 else 0
    p3 = img[y-1][x+1] if y > 0 and x < w-1 else 0
    p4 = img[y][x+1] if x < w-1 else 0
    p5 = img[y+1][x+1] if y < h-1 and x < w-1 else 0
    p6 = img[y+1][x] if y < h-1 else 0
    p7 = img[y+1][x-1] if y < h-1 and x > 0 else 0
    p8 = img[y][x-1] if x > 0 else 0
    p9 = img[y-1][x-1] if y > 0 and x > 0 else 0
    return [p2,p3,p4,p5,p6,p7,p8,p9]

def check_delete(image, y, x):
    if image[y][x] == 0:
        return False
    ps = generate_neighbour(image, y, x)
    np = len(list(filter(lambda x : x != 0, ps)))
    sp = 0
    for i in range(len(ps)):
        j = (i + 1) % len(ps)
        sp += 1 if ps[i] != 0 and ps[j] == 0 else 0
    ret = (2 <= np) and (np <= 6) and (sp == 1) and (ps[2] == 0 or ps[4] == 0 or (ps[0] == 0 and ps[6] == 0))
    return ret

def check_delete_2(image, y, x):
    if image[y][x] == 0:
        return False
    ps = generate_neighbour(image, y, x)
    np = len(list(filter(lambda x : x != 0, ps)))
    sp = 0
    for i in range(len(ps)):
        j = (i + 1) % len(ps)
        sp += 1 if ps[i] != 0 and ps[j] == 0 else 0
    ret = (2 <= np) and (np <= 6) and (sp == 1) and (ps[0] == 0 or ps[6] == 0 or (ps[2] == 0 and ps[4] == 0))
    return ret

def thinning(img):
    height, width = img.shape
    img = img.copy()
    change = True
    while change:
        must_delete = []
        change = False
        for y in range(height):
            for x in range(width):
                if check_delete(img, y, x):
                    change = True
                    must_delete += [(x,y)]
        for v in must_delete:
            img[v[1]][v[0]] = 0

        must_delete = []
        for y in range(height):
            for x in range(width):
                if check_delete_2(img, y, x):
                    change = True
                    must_delete += [(x,y)]
        for v in must_delete:
            img[v[1]][v[0]] = 0
        must_delete = []
    return img

def is_corner(img, y, x):
    neighbours = generate_neighbour(img, y, x)
    sp = 0
    for i in range(len(neighbours)):
        j = (i + 1) % len(neighbours)
        sp += 1 if neighbours[i] != 0 and neighbours[j] == 0 else 0
    return img[y][x] != 0 and sp == 1

def is_intersection(img, y, x):
    neighbours = generate_neighbour(img, y, x)
    n = sum(neighbours)
    sp = 0
    for i in range(len(neighbours)):
        j = (i + 1) % len(neighbours)
        sp += 1 if neighbours[i] != 0 and neighbours[j] == 0 else 0
    return img[y][x] != 0 and n > 2 and sp > 2

def get_corner_and_intersection(img):
    img = img.copy()
    visited = np.zeros(img.shape)
    h, w = img.shape
    res_corners = []
    res_intersections = []
    for y in range(h):
        for x in range(w):
            if img[y][x] != 0:
                corners = []
                intersections = []
                stack = [(x,y)]
                perimeter = 0
                while len(stack) > 0:
                    cx, cy = stack.pop()
                    if visited[cy][cx] == 1:
                        continue
                    perimeter += 1
                    visited[cy][cx] = 1
                    if is_corner(img, cy, cx):
                        corners.append((cx, cy))
                    if is_intersection(img, cy, cx):
                        intersections.append((cx, cy))
                    for ddx, ddy in zip(DX, DY):
                        nx = cx + ddx
                        ny = cy + ddy
                        if nx >= 0 and nx < w and ny >= 0 and ny < h and visited[ny][nx] == 0 and img[ny][nx] != 0:
                            stack.append((nx, ny))
                # remove some corners
                real_corners = []
                for i, corner in enumerate(corners):
                    real_corners += [corner]
                    for intersect in intersections:
                        # there is intersection point near this corner point
                        if np.linalg.norm(np.array(corner)-np.array(intersect)) < perimeter * 0.05:
                            real_corners.pop()
                            # remove corner until the corner are gone
                            current_corner = corner
                            s = 1
                            while s == 1:
                                img[current_corner[1]][current_corner[0]] = 0
                                ns = generate_neighbour(img, current_corner[1], current_corner[0])
                                s = 0
                                for direction_i, val in enumerate(ns):
                                    s += val
                                    if val > 0:
                                        current_corner = (
                                            current_corner[0] + DX[direction_i],
                                            current_corner[1] + DY[direction_i]
                                        )
                            break
                # recalculate intersections
                real_intersections = []
                for intersection in intersections:
                    if is_intersection(img, intersection[1], intersection[0]):
                        real_intersections += [intersection]
                if perimeter > 20:
                    res_corners += real_corners
                    res_intersections += real_intersections
                
    return res_corners, res_intersections

def get_bound(img):
    h, w = img.shape
    upper_bound = (w, h)
    lower_bound = (0, 0)
    for x in range(w):
        for y in range(h):
            if img[y][x] > 0:
                upper_bound = (min(upper_bound[0], x), min(upper_bound[1], y))
                lower_bound = (max(lower_bound[0], x + 1), max(lower_bound[1], y + 1))
    return upper_bound, lower_bound

In [5]:
def get_feature(img):
    img = cv2.erode(img, np.ones((3,3)), iterations=1)
    img = thinning(img)
    upper_bound, lower_bound = get_bound(img)
    corner, intersection = get_corner_and_intersection(img)
    
    corner_count = np.zeros(9)
    intersect_count = np.zeros(9)
    tile_density = np.zeros(9)
    
    bound_width = lower_bound[0] - upper_bound[0]
    bound_height = lower_bound[1] - upper_bound[1]
    grid_x_size = bound_width / 3
    grid_y_size = bound_height / 3
    x1, x2 = upper_bound[0] + grid_x_size, upper_bound[0] + grid_x_size * 2
    y1, y2 = upper_bound[1] + grid_y_size, upper_bound[1] + grid_y_size * 2
    
    def get_index(point):
        if point[0] < x1:
            j = 0
        elif point[0] < x2:
            j = 1
        else:
            j = 2
        if point[1] < y1:
            i = 0
        elif point[1] < y2:
            i = 1
        else:
            i = 2
        return 3 * i + j
    
    for c in corner:
        if c[0] < upper_bound[0] or c[1] < upper_bound[1] or c[0] >= lower_bound[0] or c[1] >= lower_bound[1]:
            continue
        corner_count[get_index(c)] += 1
        
    for c in intersection:
        if c[0] < upper_bound[0] or c[1] < upper_bound[1] or c[0] >= lower_bound[0] or c[1] >= lower_bound[1]:
            continue
        intersect_count[get_index(c)] += 1
        
    for x in range(upper_bound[0], lower_bound[0]):
        for y in range(upper_bound[1], lower_bound[1]):
            tile_density[get_index((x, y))] += img[y][x]
    for i in range(9):
        tile_density[i] /= (lower_bound[1] - upper_bound[1]) * (lower_bound[0] - upper_bound[0])
            
    features = []
    
    def expand(arr):
        result = []
        for h in range(1, 4):
            for w in range(1, 4):
                for y in range(3-h+1):
                    for x in range(3-w+1):
                        s = 0
                        for dx in range(w):
                            for dy in range(h):
                                s += arr[(y + dy) * 3 + (x + dx)]
                        result.append(s)
        return result
    
    features += expand(corner_count)
    features += expand(intersect_count)
    features += expand(tile_density)
    features += [(lower_bound[1] - upper_bound[1]) / (lower_bound[0] - upper_bound[0])]
            
    return features

img = cv2.imread('sample/65-0.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, img = cv2.threshold(img, 50, 255, cv2.THRESH_BINARY)
img = cv2.bitwise_not(img)

features = []
target = []

for character in range(32, 128):
    for ang in range(-3, 3):
        print('processing character: %d, ang: %d' % (character, ang))
        filename = 'sample/%d-%d.jpg' % (character, ang)
        img = cv2.imread(filename)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, img = cv2.threshold(img, 50, 255, cv2.THRESH_BINARY)
        img = cv2.bitwise_not(img)
        feature = get_feature(img)
        features.append(feature)
        target.append(chr(character))

plt.imshow(img, cmap='gray')
plt.show()

processing character: 32, ang: -3
processing character: 32, ang: -2
processing character: 32, ang: -1
processing character: 32, ang: 0
processing character: 32, ang: 1
processing character: 32, ang: 2
processing character: 33, ang: -3
processing character: 33, ang: -2
processing character: 33, ang: -1


KeyboardInterrupt: 

In [4]:
data = []
for x, y in zip(features, target):
    data.append(x + [ord(y)])
pd.DataFrame(data=data).to_csv('data.csv')